In [259]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder
from scipy.stats import chisquare, pearsonr, spearmanr
import re

import matplotlib.pyplot as plt
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


question 1 - present background
question 2 - 3 preferences of higher education (type, institute, level, major 
question 3 - how sure are you about the options you listed above 
question 4 - annual costs for studying at these institutions 
question 5 - monthly salary expectation 
question 6 - score with which you shall apply 

In [83]:
'''We load our data into a pandas dataframe. The labeling of the columns were a little hard for me to follow since 
I dont speak Spanish, so I decided to rename then which will help me follow a lot 
easier. There were specific ones that needed to be replaced, so, well, I brute forced it.'''

survey = pd.read_csv('edu_chile_survey_output.csv', header = 0, low_memory = False)
survey.rename(columns={'rut_orig':'ID', 'q1':'q1_postsec', 'q2_tipo_1_orig':'q2_institute_type_1', 
                       'q2_tipo_2_orig':'q2_institute_type_2', 'q2_tipo_3_orig':'q2_institute_type_3', 
                       'q2_nivel_1':'q2_level_1', 'q2_nivel_2':'q2_level_2', 'q2_nivel_3':'q2_level_3', 
                       'q2_carerra_1':'q2_career_1', 'q2_carerra_2':'q2_career_2', 'q2_carerra_3':'q2_career_3', 
                       'q4_nose_1_orig':'q4_IDK_1', 'q4_nose_2_orig':'q4_IDK_2', 'q4_nose_3_orig':'q4_IDK_3', 
                       'q4_cost_1_orig':'q4_TuitionEstimate_1', 'q4_cost_2_orig':'q4_TuitionEstimate_2', 
                       'q4_cost_3_orig':'q4_TuitionEstimate_3',  'q5_mi_ing_1_orig':'q5_ownwages_1', 
                       'q5_mi_ing_2_orig':'q5_ownwages_2', 'q5_mi_ing_3_orig':'q5_ownwages_3', 
                       'q5_tip_ing_1_orig':'q5_typwages_1', 'q5_tip_ing_2_orig':'q5_typwages_2', 
                       'q5_tip_ing_3_orig':'q5_typwages_3', 'q6_math_orig':'q6_math', 
                       'q5_tip_nose_1_orig': 'q5_IDK_own_1', 'q5_tip_nose_2_orig': 'q5_IDK_own_2', 
                       'q5_tip_nose_3_orig': 'q5_IDK_own_3', 'q5_mi_nose_1_orig': 'q5_IDK_typ_1',
                       'q5_mi_nose_2_orig': 'q5_IDK_typ_2', 'q5_mi_nose_3_orig': 'q5_IDK_typ_3',
                       'q6_lang_orig': 'q6_lang','PSU_leng_2013': 'lang_score', 'PSU_mate_2013': 'math_score', 
                       'PSU_2013':'comp_score','SIMCEMath10':'math_10', 'SIMCELang10':'lang_10', 
                       'mom_educ_simce':'educ_mom', 'dad_educ_simce':'educ_dad'
                      }, inplace=True)

(survey.columns.values)

array(['ID', 'start_date', 'completed_date', 'q1_postsec',
       'q2_institute_type_1', 'q2_level_1', 'q2_inst_1', 'q2_career_1',
       'q2_institute_type_2', 'q2_level_2', 'q2_inst_2', 'q2_career_2',
       'q2_institute_type_3', 'q2_level_3', 'q2_inst_3', 'q2_career_3',
       'q3', 'q4_TuitionEstimate_1', 'q4_IDK_1', 'q4_TuitionEstimate_2',
       'q4_IDK_2', 'q4_TuitionEstimate_3', 'q4_IDK_3', 'q5_ownwages_1',
       'q5_IDK_typ_1', 'q5_typwages_1', 'q5_IDK_own_1', 'q5_ownwages_2',
       'q5_IDK_typ_2', 'q5_typwages_2', 'q5_IDK_own_2', 'q5_ownwages_3',
       'q5_IDK_typ_3', 'q5_typwages_3', 'q5_IDK_own_3', 'q6_math',
       'q6_lang', 'search1_psu_math', 'search1_psu_lang', 'search1_area',
       'search1_nivel', 'search1_carrer', 'search1_clicked_button',
       'search2_psu_math', 'search2_psu_lang', 'search2_area',
       'search2_nivel', 'search2_carrer', 'search2_clicked_button',
       'search3_psu_math', 'search3_psu_lang', 'search3_area',
       'search3_nivel', 'search

In [84]:
pd.set_option('display.max_columns', None)
survey[]

,ID,start_date,completed_date,q1_postsec,q2_institute_type_1,q2_level_1,q2_inst_1,q2_career_1,q2_institute_type_2,q2_level_2,q2_inst_2,q2_career_2,q2_institute_type_3,q2_level_3,q2_inst_3,q2_career_3,q3,q4_TuitionEstimate_1,q4_IDK_1,q4_TuitionEstimate_2,q4_IDK_2,q4_TuitionEstimate_3,q4_IDK_3,q5_ownwages_1,q5_IDK_typ_1,q5_typwages_1,q5_IDK_own_1,q5_ownwages_2,q5_IDK_typ_2,q5_typwages_2,q5_IDK_own_2,q5_ownwages_3,q5_IDK_typ_3,q5_typwages_3,q5_IDK_own_3,q6_math,q6_lang,search1_psu_math,search1_psu_lang,search1_area,search1_nivel,search1_carrer,search1_clicked_button,search2_psu_math,search2_psu_lang,search2_area,search2_nivel,search2_carrer,search2_clicked_button,search3_psu_math,search3_psu_lang,search3_area,search3_nivel,search3_carrer,search3_clicked_button,search4_psu_math,search4_psu_lang,search4_area,search4_nivel,search4_carrer,search4_clicked_button,search5_psu_math,search5_psu_lang,search5_area,search5_nivel,search5_carrer,search5_clicked_button,search6_psu_math,search6_psu_lang,search6_area,search6_nivel,search6_carrer,search6_clicked_button,search7_psu_math,search7_psu_lang,search7_area,search7_nivel,search7_carrer,search7_clicked_button,search8_psu_math,search8_psu_lang,search8_area,search8_nivel,search8_carrer,search8_clicked_button,search9_psu_math,search9_psu_lang,search9_area,search9_nivel,search9_carrer,search9_clicked_button,search10_psu_math,search10_psu_lang,search10_area,search10_nivel,search10_carrer,search10_clicked_button,rbd,female,lang_score,math_score,comp_score,math_10,lang_10,educ_mom,educ_dad,schl_type,rbdRating
0,10000460631,2012-11-20 15:37:48,2012-11-20 15:38:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8485,0,583,691,637.0,1.216621,0.578311,technical school or some college,technical school or some college,Public,D
1,10000421312,2012-12-09 15:39:08,2012-12-09 15:44:47,Postulare a una institucion de educacion super...,Universidad,Carreras Profesionales,UNIVERSIDAD DE CONCEPCION,Nutricion y Dietetica,Universidad,Carreras Profesionales,UNIVERSIDAD CATOLICA DE LA SANTISIMA CONCEPCION,Nutricion y Dietetica,Universidad,Carreras Profesionales,UNIVERSIDAD CATOLICA DE LA SANTISIMA CONCEPCION,Kinesiologia,Estoy bastante seguro(a).,$2000000,off,$1800000,off,$3500000,off,$0,on,$600000,off,$0,on,$500000,off,$0,on,$700000,off,580,600,580,600,Salud,Carreras Profesionales,Nutricion y Dietetica,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18049,1,537,539,538.0,-0.122493,1.048775,8th grade or more,high school,Voucher,C
2,10000274613,2012-11-21 14:17:17,2012-11-21 16:07:45,Postulare a una institucion de educacion super...,IP,Carreras Tecnicas,IP DUOC UC,Tecnico Dental y Asistente de Odontologia,IP,Carreras Tecnicas,CFT INACAP,Tecnico Dental y Asistente de Odontologia,CFT,Carreras Tecnicas,CFT SANTO TOMAS,Tecnico Dental y Asistente de Odontologia,Estoy absolutamente seguro(a).,$1460000,off,$1560000,off,$1530000,off,$260000,off,$0,on,$260000,off,$0,on,$260000,off,$0,on,400,450,400,450,Salud,Carreras Tecnicas,Tecnico Dental y Asistente de Odontologia,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1482,1,353,436,394.5,NaN,NaN,8th grade or more,8th grade or more,Voucher,C
3,10000276454,2012-12-27 19:16:47,2012-12-05 20:17:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,NaN,off,N

# DETERMINING THOSE WHO SEARCHED THE DATABASE 

In [94]:
'''Most important step in the program, we mark the people who searched the database and those who did not search 
the database. '''

reports = (survey[['search1_psu_math','search1_psu_lang','search1_area','search1_nivel','search1_carrer']]).dropna()
rows = list(reports.index)
len(rows)
dim = survey.shape
r = dim[0]
results = [0] * r 

for row in rows: 
    results[row] = 1

survey['survey_participant'] = results

# DATA CLEANING 

In [95]:
pd.set_option('display.max_columns', None)

'''let's initiate the data cleaning process. First, we check if there are any rows which contain only NaN values, and 
get rid of them if that is the case. 
'''
survey = survey.dropna(axis = 0, how = 'all')

In [96]:
#survey[['math_10', 'lang_10']] = survey[['math_10', 'lang_10']] + 10

The next step helps us clear and categorize some of our data. As is noticeable, there are numerical and categorical variables in our data set that need to be separated in the interest of creating and training our model. After separating these variables, we set out to clean our numerical data. Some entries such as '$4.000.000' are corrected to '4000000' by the use of regular expressions. In the next section, we divide our data into inputs and outputs (outputs implying students using the database) and then into categorical and numerical. 

In [336]:
numerical_inputs = ['q4_TuitionEstimate_1', 'q4_TuitionEstimate_2', 'q4_TuitionEstimate_3', 
                    'q5_ownwages_1', 'q5_ownwages_2', 'q5_ownwages_3',  'q5_typwages_1', 
                    'q5_typwages_2', 'q5_typwages_3', 'q6_math', 'q6_lang','lang_score', 
                    'math_score', 'comp_score', 'math_10', 'lang_10']

# , 'search1_psu_math', 'search1_psu_lang'
categorical_inputs = ['q1_postsec','q2_institute_type_1', 'q2_institute_type_2', 'q2_institute_type_3',
                      'q2_level_1', 'q2_level_2', 'q2_level_3', 'q2_career_1', 'q2_career_2', 'q2_career_3',
                      'q2_inst_1', 'q2_inst_2', 'q2_inst_3', 'q3', 'q4_IDK_1', 'q4_IDK_2', 'q4_IDK_3', 
                      'q5_IDK_own_1', 'q5_IDK_own_2', 'q5_IDK_own_3', 'q5_IDK_typ_1', 'q5_IDK_typ_2', 'q5_IDK_typ_3', 
                      'educ_mom', 'educ_dad', 'schl_type', 'rbdRating']

numerical_outputs = [ 'search1_psu_math', 'search1_psu_lang', 'search2_psu_math', 'search2_psu_lang', 
                      'search3_psu_math', 'search3_psu_lang', 'search4_psu_math', 'search4_psu_lang', 
                      'search5_psu_math', 'search5_psu_lang', 'search6_psu_math', 'search6_psu_lang', 
                      'search7_psu_math', 'search7_psu_lang', 'search8_psu_math', 'search8_psu_lang', 
                      'search9_psu_math', 'search9_psu_lang', 'search10_psu_math', 'search10_psu_lang']

categorical_outputs = ['search1_nivel', 'search1_carrer', 'search2_nivel', 'search2_carrer',
                       'search3_nivel', 'search3_carrer', 'search4_nivel', 'search4_carrer',
                       'search5_nivel', 'search5_carrer', 'search6_nivel', 'search6_carrer',
                       'search7_nivel', 'search7_carrer', 'search8_nivel', 'search8_carrer',
                       'search9_nivel', 'search9_carrer', 'search10_nivel', 'search10_carrer']

In [182]:
# In this small section, we clean our numerical input and output data. It takes a little while to run.

def clean_inputs(sub):
    cleaned_data = pd.DataFrame(sub)
    for column in sub: 
        l = []
        x = sub[column]
        for value in x:
            q = re.findall(r'\d+',str(value)) # we extract just the numbers using regular expression 
            if q ==[]: 
                l.extend([None])
                continue
            l.append(int(q[0]))
        cleaned_data[column] = l
    return cleaned_data

# we fix all the issues with our numerical inputs here 
survey[numerical_inputs]  = clean_inputs(survey[numerical_inputs])
survey[numerical_outputs]  = clean_inputs(survey[numerical_outputs])

In [183]:
'''In this section, we focus our attention towards our categorical data. first, let's find out the fraction of inputs 
 in the level, careers, and institute that is missing''' 

print 'Institute 1: ', "{0:.2f}".format(sum((survey['q2_inst_1']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Level 1: ', "{0:.2f}".format(sum((survey['q2_level_1']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Career 1: ', "{0:.2f}".format(sum((survey['q2_career_1']).isnull())/
    float(len(survey['q2_career_1']))*100),'%'
print 'Institute 2: ', "{0:.2f}".format(sum((survey['q2_inst_2']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Level 2: ', "{0:.2f}".format(sum((survey['q2_level_2']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Career 2: ', "{0:.2f}".format(sum((survey['q2_career_2']).isnull())/
    float(len(survey['q2_career_1']))*100),'%'
print 'Institute 3: ', "{0:.2f}".format(sum((survey['q2_inst_3']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Level 3: ', "{0:.2f}".format(sum((survey['q2_level_3']).isnull())/
    float(len(survey['q2_career_1']))*100),'% and Career 3: ', "{0:.2f}".format(sum((survey['q2_career_3']).isnull())/
    float(len(survey['q2_career_1']))*100),'%'

Institute 1:  0.00 % and Level 1:  0.00 % and Career 1:  0.00 %
Institute 2:  0.00 % and Level 2:  0.00 % and Career 2:  0.00 %
Institute 3:  0.00 % and Level 3:  0.00 % and Career 3:  0.00 %


In [353]:
'''We make an assumption here, that we shall later test. We noticed that about 23% of students dont have entries for 
their first choice of institute, level and career, 30% for their second, and 34% for their third, we make a 
transformation on our data. We convert those values to unknown, which in itself represents a (very valid) category 
of students who are not sure of their plans. '''

transform_categories = ['q1_postsec', 'q2_institute_type_1', 'q2_institute_type_2', 'q2_institute_type_3',
                        'q2_level_1', 'q2_level_2', 'q2_level_3', 'q2_career_1', 'q2_career_2', 
                        'q2_career_3', 'q2_inst_1', 'q2_inst_2', 'q2_inst_3', 'q3']

for column in survey[transform_categories]: 
    rows = survey[column].isnull()
    row = [i for i, x in enumerate(rows) if x]
    survey[column].loc[rows] = 'unknown'

2.0

# DIMENSIONATILY REDUCTION 

In [ ]:
'''After we generate slightly cleaner data, it's time to do some dimensionality reduction. Instead of using the large 
number of features provided, we condense to the relative ones, to help generate more accurate trees less prone to 
overfitting. To test this, we run a spearman correlation to see the correlated terms. We notice a correlation between 
the institution and type of institute, as well as level and career. I considered running a Multiple Correpondence 
Analysis (PCA for categorical data), but their were issues with the only support for that in python. Instead, while 
training the forest, I decided to test the dimensions I reduced to determine the best fit for the model. '''

categories = ['q2_institute_type_1', 'q2_institute_type_2', 'q2_institute_type_3',
                       'q2_career_1', 'q2_career_2', 'q2_career_3',
                      'q2_inst_1', 'q2_inst_2', 'q2_inst_3']

#'q1_postsec','q2_level_1', 'q2_level_2', 'q2_level_3',
observed_categories = []
for x in survey[categories]: 
    observed_categories.extend([x])
    col_1 = survey[x]
    for y in survey[categories]: 
        if x == y or y in observed_categories: 
            continue
        col_2 = survey[y]
        #print "The correlation between", x , 'and', y , 'is', spearmanr(col_1,col_2)

In [ ]:
'''On studying the correlation values generated, we notice that while in some cases there seem to be good correlation, 
the numbers are not extreme enough to justify reducing the data to fewer principal features. In our next section, we 
move onto variance analysis, where we study the variations between the two groups (of people who did and didn't use 
the database). '''

# VARIANCE ANALYSIS 

Given this is a classfication problem, one good way of solving this would be to segregate the two groups by observing 
the variations in their data. For example, if test scores are significantly higher for those who use the database than those who don't (not the case here), then test scores would be a good category to include in our model. We look at our different inputs in the section below (post secondary details, institute, career and level preferences, wages and cost estimates, test scores, and 

In [384]:
'''We break our data up into the yes and the no group. This helps us investigate how the behaviour of these two 
different groups are related. '''

yes_group = survey.ix[survey['survey_participant'] == 1,]
no_group = survey.ix[survey['survey_participant'] == 0,]

yes_rows =  yes_group.shape[0]
no_rows = no_group.shape[0]

In [410]:
from collections import Counter

def normalize_ratios(a, b): 
    for key in a: 
        if key not in b: 
            continue
        value1 = a[key]
        value2 = b[key]
        a[key] = round(value1/float(yes_rows),3)
        b[key] = round(value2/float(no_rows),3)
    return a, b 

IDK_groups = ['q4_IDK_1', 'q4_IDK_2', 'q4_IDK_3', 'q5_IDK_own_1', 'q5_IDK_own_2', 
              'q5_IDK_own_3','q5_IDK_typ_1', 'q5_IDK_typ_2', 'q5_IDK_typ_3']

for column in categorical_inputs: 
    if column in ['q1_postsec','q2_career_1', 'q2_career_2', 'q2_career_3', 'q2_inst_1', 'q2_inst_2', 'q2_inst_3']: 
        continue
    y_IDK = yes_group[column]
    n_IDK = no_group[column]    
    y_IDK = dict(Counter(y_IDK))
    n_IDK = dict(Counter(y_IDK))
    y_IDK, n_IDK = normalize_ratios(y_IDK, n_IDK)
    print y_IDK, n_IDK

{'IP': 0.089, 'CFT': 0.038, 'Universidad': 0.871, 'unknown': 0.002} {'IP': 0.04, 'CFT': 0.017, 'Universidad': 0.39, 'unknown': 0.001}
{'IP': 0.103, 'CFT': 0.04, 'Universidad': 0.797, 'unknown': 0.059} {'IP': 0.046, 'CFT': 0.018, 'Universidad': 0.357, 'unknown': 0.027}
{'unknown': 0.102, 'CFT': 0.054, 'Universidad': 0.719, 'IP': 0.125} {'unknown': 0.046, 'CFT': 0.024, 'Universidad': 0.322, 'IP': 0.056}
{'Carreras Profesionales': 0.904, 'Carreras Tecnicas': 0.094, 'unknown': 0.002} {'Carreras Profesionales': 0.405, 'Carreras Tecnicas': 0.042, 'unknown': 0.001}
{'Carreras Profesionales': 0.848, 'Carreras Tecnicas': 0.092, 'unknown': 0.059} {'Carreras Profesionales': 0.38, 'Carreras Tecnicas': 0.041, 'unknown': 0.027}
{'Carreras Profesionales': 0.775, 'Carreras Tecnicas': 0.123, 'unknown': 0.102} {'Carreras Profesionales': 0.347, 'Carreras Tecnicas': 0.055, 'unknown': 0.046}
{'Estoy medianamente seguro(a).': 0.23, 'Estoy bastante seguro(a).': 0.355, 'Estoy un poco seguro(a).': 0.063, 'unkn

In [358]:
# test scores analysis 
test_cols = ['q6_math', 'q6_lang','lang_score', 'math_score', 'comp_score', 'math_10', 'lang_10']
yes_avg_scores = yes_group[test_cols].mean(axis = 0)
no_avg_scores = no_group[test_cols].mean(axis = 0)

yes_std_scores = yes_group[test_cols].std(axis = 0)
no_std_scores = no_group[test_cols].std(axis = 0)


print "Students who said Yes:", yes_avg_scores, yes_std_scores
print "Students who said No:", no_avg_scores, no_std_scores
#fig, ax = plt.subplots()
#plt.hist(yes_avg_scores)

Students who said Yes: q6_math       569.840973
q6_lang       593.319676
lang_score    547.025112
math_score    539.602281
comp_score    542.993960
math_10         0.436823
lang_10         0.425911
dtype: float64 q6_math       104.221917
q6_lang        87.474613
lang_score     94.366736
math_score     97.635317
comp_score     88.611876
math_10         0.583018
lang_10         0.575713
dtype: float64
Students who said No: q6_math       554.558063
q6_lang       576.582118
lang_score    524.563561
math_score    520.375450
comp_score    522.006528
math_10         0.391506
lang_10         0.385644
dtype: float64 q6_math       114.668827
q6_lang       102.477941
lang_score     99.818635
math_score    101.635110
comp_score     93.883752
math_10         0.562262
lang_10         0.563289
dtype: float64


In [359]:
# cost and wage analysis 
costs_wages = ['q4_TuitionEstimate_1', 'q4_TuitionEstimate_2', 'q4_TuitionEstimate_3', 
                    'q5_ownwages_1', 'q5_ownwages_2', 'q5_ownwages_3',  'q5_typwages_1', 
                    'q5_typwages_2', 'q5_typwages_3']

'''Too much overlap to use these variables to tell the data apart.'''

for cw in costs_wages:  
    yes_avg_wages = yes_group[cw].mean(axis = 0)
    no_avg_wages = no_group[cw].mean(axis = 0)
    yes_std_wages = (yes_group[cw].std(axis = 0))
    no_std_wages = no_group[cw].std(axis = 0)
    print "YES GROUP AVG:", cw, "{0:.2f}".format(yes_avg_wages), "YES GROUP SD:", cw, "{0:.2f}".format(yes_std_wages)
    print "NO GROUP AVG:", cw, "{0:.2f}".format(no_avg_wages), "NO GROUP SD:", cw, "{0:.2f}".format(no_std_wages)


YES GROUP AVG: q4_TuitionEstimate_1 2883653.95 YES GROUP SD: q4_TuitionEstimate_1 56508520.77
NO GROUP AVG: q4_TuitionEstimate_1 2378523.38 NO GROUP SD: q4_TuitionEstimate_1 36736949.68
YES GROUP AVG: q4_TuitionEstimate_2 2632125.50 YES GROUP SD: q4_TuitionEstimate_2 62064443.47
NO GROUP AVG: q4_TuitionEstimate_2 1842708.87 NO GROUP SD: q4_TuitionEstimate_2 34819218.23
YES GROUP AVG: q4_TuitionEstimate_3 2074710.22 YES GROUP SD: q4_TuitionEstimate_3 49652240.78
NO GROUP AVG: q4_TuitionEstimate_3 1533628.64 NO GROUP SD: q4_TuitionEstimate_3 17047901.08
YES GROUP AVG: q5_ownwages_1 875300.04 YES GROUP SD: q5_ownwages_1 16060611.65
NO GROUP AVG: q5_ownwages_1 630632.31 NO GROUP SD: q5_ownwages_1 8080225.91
YES GROUP AVG: q5_ownwages_2 755283.64 YES GROUP SD: q5_ownwages_2 12828814.77
NO GROUP AVG: q5_ownwages_2 497975.76 NO GROUP SD: q5_ownwages_2 968368.13
YES GROUP AVG: q5_ownwages_3 736541.78 YES GROUP SD: q5_ownwages_3 13563071.99
NO GROUP AVG: q5_ownwages_3 458352.61 NO GROUP SD: q5_

In [345]:
'''After we generate slightly cleaner data, it's time to do some dimensionality reduction. Instead of using the large 
number of features provided, we condense to the relative ones, to help generate more accurate trees less prone to 
overfitting. To test this, we run a spearman correlation to see the correlated terms. We notice a correlation between 
the institution and type of institute, as well as level and career. I considered running a Multiple Correpondence 
Analysis (PCA for categorical data), but their were issues with the only support for that in python. Instead, while 
training the forest, I decided to test the dimensions I reduced to determine the best fit for the model. '''

categories = ['q2_institute_type_1', 'q2_institute_type_2', 'q2_institute_type_3',
                       'q2_career_1', 'q2_career_2', 'q2_career_3',
                      'q2_inst_1', 'q2_inst_2', 'q2_inst_3']

#'q1_postsec','q2_level_1', 'q2_level_2', 'q2_level_3',
observed_categories = []
for x in survey[categories]: 
    observed_categories.extend([x])
    col_1 = survey[x]
    for y in survey[categories]: 
        if x == y or y in observed_categories: 
            continue
        col_2 = survey[y]
        #print "The correlation between", x , 'and', y , 'is', spearmanr(col_1,col_2)

In [361]:
from sklearn.decomposition import PCA

'''Then, we consider inputs that have test scores. Unlike categorical inputs (discussed later), we drop the students 
who input no form of score. It makes it harder to come up with numerical estimates of what the data maybe, and often 
we land up having the data tell us what we want to hear, instead of what its truly trying to say. To minimize the 
number of students that are dropped, we conduct a correlation study between the expected, actual and 10th grade test 
scores. The highly correlated variables can be condensed to one through an understanding of dimensionality reduction. 
'''
math_test_scores = ['q6_math','math_score', 'math_10' ] # ,'math_10', 'lang_10'
lang_test_scores = ['q6_lang','lang_score', 'lang_10']
math_scores = survey[math_test_scores].dropna(axis = 0, how = 'any')
lang_scores = survey[lang_test_scores].dropna(axis = 0, how = 'any')

x = (survey[numerical_inputs].dropna())
final_rows = list(x.index)
x.index = range(0,len(final_rows))


'''We condense the size of the scores, to reduce the number of dimensions in final model by running a PCA. Selecting 
2 components explains about 87% of the variation, and 3 components explain about 97% of the variations. We test 
each of these in our final model and chose the best fit.'''
pca = PCA(n_components = 3)
X = pca.fit(x)
X = pd.DataFrame(pca.transform(x))

In [362]:
'''Now we turn our attention towards implementing and training our model. With the data that we have, there are 
two approaches that we can take - Random Forests or the Naive Bayes Classifier. Since the final implementation after 
curing the data isn't too complicated, we test out both to see which one works better! But before we implement then, 
there's that crucial step of breaking the data in training and testing sets to test our final models.
'''

from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score


final_survey = survey.ix[final_rows,]
default_rows = range(0,len(final_survey))
final_survey.index = default_rows

# 'search1_area', 'search1_nivel', 'search1_carrer'
features = ['q1_postsec', 'q2_institute_type_1', 'q2_level_1', 'q2_career_1', 'q2_institute_type_2', 'q2_level_2','q2_career_2',
           'q2_institute_type_3', 'q2_level_3','q2_career_3', 'q3']


#final_survey['isTrain'] = np.random.uniform(0,1,len(final_survey)) <= 0.75
report = pd.concat([final_survey[features],x],axis = 1, ignore_index = True)
report_columns = features + numerical_inputs
report.columns = report_columns
report['survey_participant'] = final_survey['survey_participant'] 

In [363]:
sum(report['survey_participant'])

7164

In [364]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler

le = sklearn.preprocessing.LabelEncoder()
nm = sklearn.preprocessing.StandardScaler()
variables = pd.DataFrame()

for column in features: 
    #print column
    x = report[column]
    if column == 'survey_participant': 
        continue
    if column not in numerical_inputs: 
        le.fit(x)
        var = pd.DataFrame(le.transform(x))
        variables = pd.concat([variables,var], axis=1)
    else:    
        nm.fit(x)
        var = pd.DataFrame(nm.transform(x))
        variables = pd.concat([variables,var], axis=1)

In [365]:
variables.dropna()
variables.shape

(15569, 11)

In [366]:
from sklearn.metrics import precision_recall_fscore_support as pr
def precisionRecall(x,y):
    # x is true positive 
    # x - y = 1 is false negative 
    # y - x = 1 is false positive 
    tp = x 
    fp = [sum(z)  for z in y-x if z>0]
    sum_y = sum(x)
    sum_yhat = sum(y)
    print sum_y/float(sum_y + sum_yhat)
    
    
    

In [417]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import GradientBoostingClassifier

gdc = GradientBoostingClassifier()
lr = LogisticRegression()
clf = svm.SVR()
et = ExtraTreesClassifier()
rgr = RadiusNeighborsRegressor()
forest = RandomForestRegressor(n_estimators = 100, n_jobs = 2, oob_score=True)
adaboost = AdaBoostRegressor()
nb = GaussianNB()
rd = RidgeClassifierCV()
kf = KFold(report.shape[0], n_folds = 5)

for train_index, test_index in kf:
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = variables.ix[list(train_index),], variables.ix[list(test_index),]
    y_train = report['survey_participant'].ix[list(train_index),]
    y_test = report['survey_participant'].ix[list(test_index),]
    forest.fit(X_train,y_train)
    adaboost.fit(X_train,y_train)
    gdc.fit(X_train, y_train)
    rd.fit(X_train, y_train)
    rgr.fit(X_train, y_train)
    nb.fit(X_train, y_train)
    lr.fit(X_train, y_train)
    et.fit(X_train, y_train)
    #print forest.feature_importances_
    y_hat = list(gdc.predict(X_test))
    print 'GDC', sum((y_hat-y_test)**2)/float(len(y_test))
    y_hat = list(rd.predict(X_test))
    print 'RD', sum((y_hat-y_test)**2)/float(len(y_test))
    y_hat = list(et.predict(X_test))
    print 'ET', sum((y_hat-y_test)**2)/float(len(y_test))
    y_hat = list(lr.predict(X_test))
    print 'LR', sum((y_hat-y_test)**2)/float(len(y_test))
    y_hat = list(forest.predict(X_test))
    print 'RFRegressor', sum(((y_hat)-y_test)**2)/float(len(y_test))
   

 GDC 0.454078355812
RD 0.448298008992
ET 0.473346178548
LR 0.447655748234
RFRegressor 0.278316307517
GDC 0.452793834297
RD 0.464675658317
ET 0.467244701349
LR 0.464354527938
RFRegressor 0.27529119795
GDC 0.471740526654
RD 0.469492614001
ET 0.476557482338
LR 0.469492614001
RFRegressor 0.279422368698
GDC 0.454399486191
RD 0.452472703918
ET 0.481695568401
LR 0.45183044316
RFRegressor 0.278329131516
GDC 0.470285897848
RD 0.472213299068
ET 0.484420173466
LR 0.472534532605
RFRegressor 0.285430118461


In [327]:
x = y_test
y = y_hat
tp = x 
fp = [sum(z)  for z in y-x if z>0]
sum_y = sum(x)
sum_yhat = sum(y)
print y_hat
#print sum_y/float(sum_y + sum_yhat)

[nan, nan, 0.52272727272727271, nan, nan, 0.52272727272727271, 0.5625, 0.0, 0.40909090909090912, 0.0, nan, 0.0, 0.55555555555555558, nan, nan, 0.25, 1.0, nan, 0.40000000000000002, 0.40845070422535212, 0.0, 0.0, 0.0, 0.48837209302325579, 0.52272727272727271, nan, 0.54929577464788737, 0.52054794520547942, 0.42307692307692307, nan, 0.5, 0.59999999999999998, 0.52272727272727271, nan, 0.61111111111111116, 0.61538461538461542, 0.5, nan, 0.44444444444444442, 0.26315789473684209, nan, 0.52272727272727271, 0.40000000000000002, 0.52272727272727271, 0.52112676056338025, 0.30769230769230771, nan, nan, nan, 1.0, nan, nan, 1.0, nan, 0.48695652173913045, 0.0, 0.54545454545454541, 0.0, nan, 0.63636363636363635, 1.0, 0.40579710144927539, 0.47826086956521741, nan, 0.5, 1.0, nan, nan, nan, 0.0, 1.0, nan, nan, nan, 0.53703703703703709, 0.2857142857142857, nan, 1.0, 0.58333333333333337, 0.46090534979423869, nan, 0.0, nan, 1.0, nan, 0.55769230769230771, 0.52112676056338025, 0.52272727272727271, 0.0, nan, 0.

Now we break our data into testing and training sets. Later on, we shall deal with this by using a CV method 

In [72]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(variables, report['survey_participant'][report['isTrain']== True])
y_hat = list(clf.predict(predictors))
sum((y_hat-y)**2)/float(len(y))

0.46903900281463612

In [101]:
# now that we have our data without NA values in report, let us vectorize the terms to use in our Random Forest 
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer
forest = RandomForestClassifier(n_estimators = 100, n_jobs = 2,oob_score=True)

#,'q5_tip_ing_1_orig','q5_tip_ing_2_orig','q5_tip_ing_3_orig'
#pd.DataFrame(train[['q6_math_orig','q6_lang_orig']],x)

le = sklearn.preprocessing.LabelEncoder()
variables = pd.DataFrame()
predictors = pd.DataFrame()
for column in train: 
    x = report[column].dropna()
    le.fit(x)
    var = pd.DataFrame(le.transform(train[column].dropna()))
    variables = pd.concat([variables,var], axis=1)
    pred = pd.DataFrame(le.transform(test[column].dropna()))
    predictors = pd.concat([predictors,pred], axis=1)

#variables = pd.concat([variables,report[numerical]], axis = 1)

forest.fit(variables,report['survey_participant'][report['isTrain'] == True])

pred = pd.concat([pred1,pred2,pred3,pred4,pred5],axis=0)

y_hat = list(forest.predict(predictors))
y = report['survey_participant'][report['isTrain'] == False]
sum((y_hat-y)**2)


NameError: name 'pred1' is not defined

In [165]:
reports = (survey[['search1_psu_math','search1_psu_lang','search1_area','search1_nivel','search1_carrer']]).dropna()
rows = list(reports.index)
len(rows)
dim = survey.shape
r = dim[0]
results = [0] * r 
for row in rows: 
    results[row] = 1
survey['survey_participant'] = results



We can't use the mean strategy because it doesn't work well for something like missing wages, where there is a large variation based on factors like school type, career and level (making up data can be a bad choice because we often shift our data set towards what we want it to tell us - which may be different from what it actually tells us. 

In [133]:
clf = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = 100)
y = survey['survey_participant']
x = survey.ix[:,3:37]

#scores = clf.fit(x,y)

In [ ]:
for column in sparse_matrix: 
    print column

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 82)	1
  (0, 83)	1
  (0, 84)	1
  (0, 85)	1
  (0, 86)	1
  (0, 87)	1
  (0, 88)	1
  (0, 89)	1
  (0, 90)	1
  (0, 91)	1
  (0, 92)	1
  (0, 93)	1
  (0, 94)	1
  (0, 95)	1
  (0, 96)	1
  (0, 97)	1
  (0, 99)	1
  (0, 100)	1
  (0, 101)	1
  (0, 102)	1
  (0, 103)	1
  (0, 104)	1
  (0, 105)	1
  (0, 106)	1
  (0, 107)	1
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 24)	1
  (0, 25)	1
  :	:
  (0, 84)	1
  (0, 85)	1
  (0, 86)	1
  (0, 87)	1
  (0, 88)	1
  (0, 89)	1
  (0, 90)	1
  (0, 91)	1
  (0

In [ ]:
def cross_val(X, y, clf_class, **kwargs):
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_pred = y.copy()
   
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train, y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred
# Checks the accuracy of the algorithm
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

In [132]:
'''non parametric 
dimensionality reduction 
'''